# Introduction to Probabilisitic Methods Exercises

**2025 Introduction to Quantiative Methods in Finance**

**The Erdös Institute**


These are exercises for practice, not to be submitted.

## Exercise 1 ## 

Design and code a probabilistic method to estimate $\pi\sim 3.14$.

In [96]:
import numpy as np
from scipy import random as rand
from IPython.display import display, Latex

In [100]:
N = 500000 # samples
R_sq_circ = 1.

in_circ = 0
display(Latex(r'Our estimate for $\pi$ is...... **drumrollllll*:'))
for _ in range(N):
    x = rand.uniform()
    y = rand.uniform()
    if(x*x + y*y <= R_sq_circ):
        in_circ += 1

pi = 4*in_circ/N
print(pi)
    

<IPython.core.display.Latex object>

3.1444


## Exercise 2 ## 

###  Rules of Craps (Vegas Casino Style)

 
The basic rules are outlined below and a several common betting options are given.

### 1. The Come-Out Roll
 - The game begins with the shooter making a "come-out" roll using two six-sided dice.
 - If the come-out roll is a **7 or 11**, the pass line bet wins (called a "natural").
 - If the come-out roll is **2, 3, or 12**, the pass line bet loses (called "craps").
 - If the come-out roll is **4, 5, 6, 8, 9, or 10**, that number becomes the **point**.

### 2. Point Phase
 - Once the point is established, the shooter continues rolling.
 - The goal is to roll the **point number again before rolling a 7**.
     - If the shooter rolls the point again, pass line bets win.
     - If the shooter rolls a **7 before the point**, pass line bets lose (called "seven out").

### 3. Some Betting Options
 - **Pass Line Bet**: Wins on a natural (7 or 11), loses on craps (2, 3, 12), wins if point is hit before a 7.
 - **Don't Pass Bet**: Opposite of pass line. Wins on craps (2, 3), ties on 12, loses on 7 or 11.

 
 
 ### Part 1
 Determine the odds of winning at craps for the Pass Line Bet and Don't Pass Bet.
 
 ### Part 2
 Use probabilistic/Monte-Carlo methods for estimating the odds of winning the Pass Line and Don't Pass Line Bets.

## Odds of winning Pass Line Bet

### Probability of winning the first phase is the probability of rolling:
    - A) a seven or eleven
    - B) a point roll combined with rolling it again first (before a seven)
    
Probabilitly of A $= P(7) + P(11) = \frac{6}{36} + \frac{2}{36}$  
Probability of B $= \sum_{i=4;\neq7}^{10} P(i)\frac{P(i)}{P(i) + P(7)}$  
Probability of $A \cup B \approx .4929$

## Odds of winning Don't Pass Line Bet

### Probability of winning the first phase is the probability of rolling:
    - A) a two or three
    - B) a point roll combined with rolling a 7 first (before rolling the point again)
    
Probabilitly of A $= P(2) + P(3) = \frac{1}{36} + \frac{2}{36}$  
Probability of B $= \sum_{i=4;\neq7}^{10} P(i)\frac{P(7)}{P(i) + P(7)}$  
Probability of $A \cup B \approx .4793$

## Monte-Carlo'ing the answer

In [123]:
def Roll():
    #for whatever diabolical reason randint high value is NON-INCLUSIVE
    d1 = rand.randint(1,7)
    d2 = rand.randint(1,7)
    return d1+d2
def Craps():
    roll = Roll()
    if (roll==7) or (roll==11):
        return np.array([1,0])
    if (roll==2) or (roll==3):
        return np.array([0,1])
    if(roll==12):
        return np.array([0,0])
    # if these conditions aren't met, we move on the second phase
    # i.e. must have rolled a point
    newRoll = Roll()
    while not(newRoll == roll or newRoll == 7):
        newRoll = Roll()
    if newRoll == roll:
        return np.array([1,0])
    else:
        return np.array([0,1])
    
    
#Number of Games: N

N = 500000
win_count = np.array([0,0])
print("Let's go to Vegas!!!!")
print("...")
for _ in range(N):
    win_count += Craps()
    
print("Pass win probability: ", win_count[0]/N)
print("Don't Pass win probability: ", win_count[1]/N)


Let's go to Vegas!!!!
...
Pass win probability:  0.493068
Don't Pass win probability:  0.478672


## Exercise 3 ##

Ten cards labeled 1 through 10 are shuffled. What is the probability that one of the ten cards was shuffled to its original location?

Find the true expectation and try using a probabilistic method to estimate the answer.

In [158]:
N = 50000
card_deck = [i for i in range(1,11)]
new_deck = []
miraculous_cards = 0
expectation = 0
print("Pick a card.. any card.... can be.. in its.. originial.. positio..n ...")
for _ in range(N):
    while card_deck:
        draw_from = rand.randint(0,len(card_deck))
        draw = card_deck.pop(draw_from)
        new_deck.append(draw)
        if draw == len(new_deck):
            miraculous_cards += 1
    expectation += miraculous_cards
    card_deck = [i for i in range(1,11)]
    new_deck = []
    miraculous_cards = 0
print("Expectation of \"miraculous\" cards: ", expectation/N)

Pick a card.. any card.... can be.. in its.. originial.. positio..n ...
Expectation of "miraculous" cards:  1.00398


## Exercise 4

You approach two slot machines. Each machine costs $\$.050$ to play and pays out $\$1$ for a win. You do not know the exact probability of winning on either machine, but you are told that **one has a 2/5 chance of winning**, and the **other has a 3/5 chance of winning**. You do not know which machine has which probability.

Your goal is to use probabilistic methods to compare two strategies over a total of 100 plays, to see which strategy is more profitable on average. A more profitable strategy is one that tends to allocate more plays to the machine with the higher payout probability.

### Strategies to Simulate

1. **Loss-Switch Strategy:**  
   Select a slot machine at random. Play on that machine until you lose, then switch to the other machine. Repeat this process for 100 plays.

2. **Exploration-Exploitation Strategy:**  
   Start by playing each machine 10 times. Estimate the win rate for each machine based on these initial plays. Then:
   - Continue playing the machine with the higher observed win rate.
   - After each play, update the estimated win rate.
   - If the other machine’s estimated win rate becomes higher, switch to it.
   - Repeat until 100 total plays have been made.

---

**Note:**  
While this is a simplified, hypothetical scenario, it mirrors real-world decision-making under uncertainty. For instance, in finance, a trader might have multiple strategies available for a given asset. The decision of which strategy to follow at a given time can be based on adaptive rules informed by past performance, similar in spirit to the strategies explored here.